In [1]:
import pandas as pd
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [2]:
lines = pd.read_csv('fra.txt', names=['src', 'tar', 'lic'], sep='\t')

In [3]:
del lines['lic']

In [4]:
lines

,src,tar
0,Go.,Va !
1,Hi.,Salut !
2,Hi.,Salut.
3,Run!,Cours !
4,Run!,Courez !
...,...,...
179899,"Top-down economics never works, said Obama. ""T...","« L'économie en partant du haut vers le bas, ç..."
179900,A carbon footprint is the amount of carbon dio...,Une empreinte carbone est la somme de pollutio...
179901,Death is something that we're often discourage...,La mort est une chose qu'on nous décourage sou...
179902,Since there are usually multiple websites on a...,Puisqu'il y a de multiples sites web sur chaqu...


In [5]:
lines = lines.loc[:, 'src':'tar']
lines = lines[0:50000] # 5만개만 저장
lines.sample(10)

,src,tar
10530,Are you at home?,Tu es chez toi ?
28358,What's there to do?,Qu'y a-t-il à faire ?
3758,I can't sing.,Je suis incapable de chanter.
26406,It's time to party.,Il est l'heure de faire la fête.
30812,I knew that already.,Je le savais déjà.
13481,Tom's miserable.,Tom est malheureux.
12447,It's not pretty.,Ce n'est pas joli.
12464,It's quite safe.,C'est plutôt pépère.
12314,It looked awful.,Ça avait l'air atroce.
13646,We're all angry.,Nous sommes toutes en colère.


In [6]:
lines.tar = lines.tar.apply(lambda x : '\t '+ x + ' \n') #<sos> 와 <eos> 추가
lines.sample(10)

,src,tar
31305,I will not help you.,\t Je ne t'aiderai pas. \n
22707,We miss you a lot.,\t Vous nous manquez beaucoup. \n
33050,They have a problem.,\t Elles ont un problème. \n
494,Stop Tom.,\t Stoppez Tom. \n
7558,You're insane.,\t Tu es cinglé. \n
21185,It's worth a trip.,\t Ça vaut le voyage. \n
16601,Let's have sushi.,\t Prenons des sushi. \n
48052,Can you sing us a song?,\t Pouvez-vous nous chanter une chanson ? \n
15915,I was humiliated.,\t On m'a humilié. \n
6850,That's a risk.,\t C'est un risque. \n


In [7]:
# 글자 집합 구축
src_vocab=set()
for line in lines.src: # 1줄씩 읽음
    for char in line: # 1개의 글자씩 읽음
        src_vocab.add(char)

tar_vocab=set()
for line in lines.tar:
    for char in line:
        tar_vocab.add(char)

In [8]:
src_vocab_size = len(src_vocab)+1
tar_vocab_size = len(tar_vocab)+1
print(src_vocab_size) # 영어: 77 글자
print(tar_vocab_size) # 불어:104 글자

77
104


In [9]:
src_vocab=sorted(list(src_vocab))
tar_vocab = sorted(list(tar_vocab))

In [10]:
src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])
print(src_to_index)
print(tar_to_index)

{' ': 1, '!': 2, '"': 3, '$': 4, '%': 5, '&': 6, "'": 7, ',': 8, '-': 9, '.': 10, '0': 11, '1': 12, '2': 13, '3': 14, '4': 15, '5': 16, '6': 17, '7': 18, '8': 19, '9': 20, ':': 21, '?': 22, 'A': 23, 'B': 24, 'C': 25, 'D': 26, 'E': 27, 'F': 28, 'G': 29, 'H': 30, 'I': 31, 'J': 32, 'K': 33, 'L': 34, 'M': 35, 'N': 36, 'O': 37, 'P': 38, 'Q': 39, 'R': 40, 'S': 41, 'T': 42, 'U': 43, 'V': 44, 'W': 45, 'X': 46, 'Y': 47, 'Z': 48, 'a': 49, 'b': 50, 'c': 51, 'd': 52, 'e': 53, 'f': 54, 'g': 55, 'h': 56, 'i': 57, 'j': 58, 'k': 59, 'l': 60, 'm': 61, 'n': 62, 'o': 63, 'p': 64, 'q': 65, 'r': 66, 's': 67, 't': 68, 'u': 69, 'v': 70, 'w': 71, 'x': 72, 'y': 73, 'z': 74, 'é': 75, '€': 76}
{'\t': 1, '\n': 2, ' ': 3, '!': 4, '"': 5, '$': 6, '%': 7, '&': 8, "'": 9, '(': 10, ')': 11, ',': 12, '-': 13, '.': 14, '0': 15, '1': 16, '2': 17, '3': 18, '4': 19, '5': 20, '6': 21, '7': 22, '8': 23, '9': 24, ':': 25, '?': 26, 'A': 27, 'B': 28, 'C': 29, 'D': 30, 'E': 31, 'F': 32, 'G': 33, 'H': 34, 'I': 35, 'J': 36, 'K': 3

In [11]:
#입력 영어 문장에 대한 정수 인코딩

In [12]:
encoder_input = []
for line in lines.src: #입력 데이터에서 1줄씩 문장을 읽음
    temp_X = []
    for w in line: #각 줄에서 1개씩 글자를 읽음
        temp_X.append(src_to_index[w]) # 글자를 해당되는 정수로 변환
    encoder_input.append(temp_X)

In [13]:
encoder_input[3]

[40, 69, 62, 2]

In [14]:
decoder_input = []
for line in lines.tar: #입력 데이터에서 1줄씩 문장을 읽음
    temp_X = []
    for w in line: #각 줄에서 1개씩 글자를 읽음
        temp_X.append(tar_to_index[w]) # 글자를 해당되는 정수로 변환
    decoder_input.append(temp_X)

In [15]:
decoder_input[:5] # 1,3 / 3,2 <EOS>

[[1, 3, 48, 53, 3, 4, 3, 2],
 [1, 3, 45, 53, 64, 73, 72, 3, 4, 3, 2],
 [1, 3, 45, 53, 64, 73, 72, 14, 3, 2],
 [1, 3, 29, 67, 73, 70, 71, 103, 4, 3, 2],
 [1, 3, 29, 67, 73, 70, 57, 78, 103, 4, 3, 2]]

In [16]:
lines

,src,tar
0,Go.,\t Va ! \n
1,Hi.,\t Salut ! \n
2,Hi.,\t Salut. \n
3,Run!,\t Cours ! \n
4,Run!,\t Courez ! \n
...,...,...
49995,I may win if I'm lucky.,\t Je peux gagner si j'ai de la chance. \n
49996,I meet him at the club.,\t Je le rencontre au club. \n
49997,I meet him at the club.,\t Je le rencontre au cercle. \n
49998,I met him in the crowd.,\t Je l'ai rencontré dans la foule. \n


In [17]:
decoder_target = []
for line in lines.tar:
    t=0
    temp_X = []
    for w in line:
        if t>0:
            temp_X.append(tar_to_index[w])
        t=t+1
    decoder_target.append(temp_X)
print(decoder_target[:5])

[[3, 48, 53, 3, 4, 3, 2], [3, 45, 53, 64, 73, 72, 3, 4, 3, 2], [3, 45, 53, 64, 73, 72, 14, 3, 2], [3, 29, 67, 73, 70, 71, 103, 4, 3, 2], [3, 29, 67, 73, 70, 57, 78, 103, 4, 3, 2]]


In [18]:
max_src_len = max([len(line) for line in lines.src])
max_tar_len = max([len(line) for line in lines.tar])
print(max_src_len)#23, 영어
print(max_tar_len) #76, 불어

23
76


In [19]:
encoder_input = pad_sequences(encoder_input, maxlen=max_src_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')

In [20]:
encoder_input[0]

array([29, 63, 10,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0], dtype=int32)

In [21]:
# 원핫 인코딩
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

In [22]:
encoder_input[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [23]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
import numpy as np

In [24]:
encoder_inputs = Input(shape=(None, src_vocab_size))
encoder_lstm = LSTM(units=256, return_state=True) 
#return_state = True : 인코더에 입력을 넣으면 내부 상태를 리턴 (cell 끼리 주고 받는 cell의 상태를 리턴)
#return_state 받은 값을 디코더에게 전달할때 필요
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
# encoder_outputs도 같이 리턴받기는 했지만 여기서는 필요없으므로 이 값은 버림.
# state_h = hidden state, state_c = cell state
encoder_states = [state_h, state_c]
# LSTM은 바닐라 RNN과는 달리 상태가 두 개. 바로 은닉 상태와 셀 상태.

In [25]:
# None = num.of vectors / tar_vocab_size = 104
decoder_inputs = Input(shape=(None, tar_vocab_size)) 
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)
decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state=encoder_states)
# 디코더의 첫 상태를 인코더의 은닉 상태, 셀 상태로 합니다.
decoder_softmax_layer = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")

In [45]:
model.fit(x=[encoder_input, decoder_input], y=decoder_target, batch_size=64, epochs=10, validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 124s 198ms/step - loss: 0.4055 - val_loss: 0.4857
Epoch 2/10
625/625 [==============================] - 117s 187ms/step - loss: 0.3606 - val_loss: 0.4476
Epoch 3/10
625/625 [==============================] - 116s 185ms/step - loss: 0.3292 - val_loss: 0.4218
Epoch 4/10
625/625 [==============================] - 122s 195ms/step - loss: 0.3062 - val_loss: 0.4029
Epoch 5/10
625/625 [==============================] - 119s 191ms/step - loss: 0.2879 - val_loss: 0.3887
Epoch 6/10
625/625 [==============================] - 118s 188ms/step - loss: 0.2732 - val_loss: 0.3791
Epoch 7/10
625/625 [==============================] - 119s 190ms/step - loss: 0.2610 - val_loss: 0.3715
Epoch 8/10
625/625 [==============================] - 119s 190ms/step - loss: 0.2502 - val_loss: 0.3659
Epoch 9/10
625/625 [==============================] - 119s 191ms/step - loss: 0.2412 - val_loss: 0.3609
Epoch 10/10
625/625 [==============================] - 127s 204m

In [27]:
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)


In [47]:
model.save("seq2seq.h5")

In [28]:
# 이전 시점의 상태들을 저장하는 텐서
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 사용. 이는 뒤의 함수 decode_sequence()에 구현
decoder_states = [state_h, state_c]
# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태인 state_h와 state_c를 버리지 않음.
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)

In [29]:
index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tar = dict((i, char) for char, i in tar_to_index.items())

In [53]:
tar_s=np.zeros((1,1,10))

In [54]:
tar_s[0,0,tar_to_index['\t']] = 1.


In [55]:
tar_s

array([[[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]]])

In [129]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1, 1, tar_vocab_size))
    target_seq[0, 0, tar_to_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ""

    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
#         print(type(output_tokens), output_tokens.shape)
        # 예측 결과를 문자로 변환
#         print(output_tokens[0, -1, :])
#         print(np.sum(output_tokens[0, -1, :]-output_tokens[0, 0, :]))
#         print('--------'*10)
        sampled_token_index = np.argmax(output_tokens[0, -1, :]) # 풀리지 않는 -1... 0 
        sampled_char = index_to_tar[sampled_token_index]

        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += sampled_char

        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_tar_len):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 시퀀스 형태로 저장
        target_seq = np.zeros((1, 1, tar_vocab_size))
        target_seq[0, 0, sampled_token_index] = 1.

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]
    return decoded_sentence

In [130]:
encoder_input.shape

(50000, 23, 77)

In [131]:
encoder_input[3:4], encoder_input[3:4].shape # 슬라이싱과 인덱싱은 서로 차원이 다르게 나온닷

(array([[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [1., 0., 0., ..., 0., 0., 0.],
         [1., 0., 0., ..., 0., 0., 0.],
         [1., 0., 0., ..., 0., 0., 0.]]], dtype=float32),
 (1, 23, 77))

In [132]:
encoder_input[3], encoder_input[3].shape # 

(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 (23, 77))

In [133]:
for seq_index in [3,50,100,300,1001]: # 입력 문장의 인덱스
    input_seq = encoder_input[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', lines.src[seq_index])
    print('정답 문장:', lines.tar[seq_index][1:len(lines.tar[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
    print('번역기가 번역한 문장:', decoded_sentence[:len(decoded_sentence)-1]) # '\n'을 빼고 출력

-----------------------------------
입력 문장: Run!
정답 문장:  Cours ! 
번역기가 번역한 문장:  Attendez-vous ! 
-----------------------------------
입력 문장: I left.
정답 문장:  Je suis parti. 
번역기가 번역한 문장:  Je le suis rentré. 
-----------------------------------
입력 문장: Burn it.
정답 문장:  Brûlez-la. 
번역기가 번역한 문장:  Appelle-le. 
-----------------------------------
입력 문장: Drive on.
정답 문장:  Continue à rouler ! 
번역기가 번역한 문장:  Appelle Tom. 
-----------------------------------
입력 문장: Step back.
정답 문장:  Recule ! 
번역기가 번역한 문장:  Prenez une boisson ! 
